In [1]:
# import time
# import copy
import wandb
import torch
import warnings
warnings.filterwarnings('ignore')
# import torch_geometric

from torch import nn, optim
from torch.nn import functional as F
from torch_geometric import nn as gnn
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import ModelNet
from IPython.display import display, clear_output
# from torch_geometric.utils import remove_self_loops
# from sklearn.model_selection import train_test_split
from torch_geometric.transforms import Compose, RandomRotate, SamplePoints, KNNGraph, NormalizeScale

import numpy as np
import matplotlib.pyplot as plt

from model import GNN, PNET, PointNet, PointViG, PointNet2


torch.manual_seed(seed=42)

In [2]:
sweep_config = {
	'method': 'random',
}

metric = {
	'name': 'val accuracy',
	'goal': 'maximize',
}

parameters = {
	'optimizer': {
		'values': ['adam']
	},
	'epochs': {
		'value': 10
	},
	'batch_size': {
		'values': [32, 64, 128, 256]
	},
	'n_samples': {
		'values': [64, 128, 256, 512]
	},
	'learning_rate': {
		'values': [5e-5, .0001, .0005, .001]
	},
	'weight_decay': {
		'values': [0, .0001]
	}

}

sweep_config['metric'] = metric
sweep_config['parameters'] = parameters

# wandb.init(
# 	project='GeoDL'
# )

In [3]:
import pprint
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'val accuracy'},
 'parameters': {'batch_size': {'values': [32, 64, 128, 256]},
                'epochs': {'value': 10},
                'learning_rate': {'values': [5e-05, 0.0001, 0.0005, 0.001]},
                'n_samples': {'values': [64, 128, 256, 512]},
                'optimizer': {'values': ['adam']},
                'weight_decay': {'values': [0, 0.0001]}}}


In [4]:
sweep_id = wandb.sweep(sweep_config, project='GeoDL')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: pqgcsuwo
Sweep URL: https://wandb.ai/technionsaleem/GeoDL/sweeps/pqgcsuwo


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def transform_input(e, n_samples):
	y = e.y
	# e = e.pos.view(int(e.pos.shape[0] / n_samples), n_samples, 3)
	# e = e.transpose(1, 2)
	return e, y

def build_dataset(batch_size, n_samples):
   
	transform = Compose([
		# random_rotate,
		# jitter,
		# shuffle,
		SamplePoints(num=n_samples, include_normals=True, remove_faces=True),
		# KNNGraph(k=6),
	])
	
	pre_transform = NormalizeScale()

	train_data = ModelNet('ModelNet/',
						name='10',
						transform=transform,
						pre_transform=pre_transform,
						train=True)
	
	val_data = ModelNet('ModelNet/',
					name='10',
					transform=transform,
					pre_transform=pre_transform,
					train=False)
	
	dataloader = DataLoader(dataset=train_data,
						batch_size=batch_size,
						shuffle=True)
	
	val_dataloader = DataLoader(dataset=val_data,
							shuffle=True,
							batch_size=batch_size)

	return dataloader, val_dataloader


def build_network():
	# network = PointNet(in_channels=3, out_channels=10)
	ratio_1, radius_1 = .748, .4817
	ratio_2, radius_2 = .3316, .2447
	dropout = .1
	network = PointNet2(ratio_1, ratio_2, radius_1, radius_2, dropout)
	return network.to(device)


def build_optimizer(model, learning_rate, weight_decay):
	optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
	return optimizer

def calc_acc(y, y_hat):
	return (y == y_hat).type(torch.float).mean().detach().cpu().item()

def calc_val_acc(m, val_dataloader, n_samples):
	m.eval()

	val_acc = []
	for obj in val_dataloader:
		obj = obj.to(device)
		obj, y = transform_input(obj, n_samples)

		o = m(obj)
		y_hat = o.argmax(dim=-1)

		val_acc.append(calc_acc(y, y_hat))
	
	val_acc = sum(val_acc) / len(val_acc)
	return val_acc

def train_epoch(network, loader, val_loader, optimizer, n_samples, epoch):
	criterion = nn.CrossEntropyLoss()
	cumu_loss = 0
	train_acc = []

	network.train()
	for _, e in enumerate(loader):
		e = e.to(device)

		e, y = transform_input(e, n_samples)

		optimizer.zero_grad()
		o = network(e)

		# ➡ Forward pass
		loss = criterion(o, y)
		cumu_loss += loss.item()

		# ⬅ Backward pass + weight update
		loss.backward()
		optimizer.step()

		train_acc.append(calc_acc(y, o.argmax(dim=-1)))

		wandb.log({"batch loss": loss.item()})
		wandb.log({"batch train accuracy": train_acc[-1]})
	
	
	wandb.log({"train accuracy": sum(train_acc) / len(train_acc), "epoch": epoch})
	wandb.log({"val accuracy": calc_val_acc(network, val_loader, n_samples), "epoch": epoch})


	return cumu_loss / len(loader)

In [6]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms



def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        loader, val_loader = build_dataset(config.batch_size, config.n_samples)
        network = build_network()
        optimizer = build_optimizer(network, config.learning_rate, config.weight_decay)

        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, loader, val_loader, optimizer, config.n_samples, epoch)
            
            wandb.log({"loss": avg_loss,
                       'epoch': epoch})

In [7]:
wandb.agent(sweep_id, train, count=10)

wandb: Agent Starting Run: 6khbaega with config:
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	learning_rate: 0.0001
wandb: 	n_samples: 64
wandb: 	optimizer: adam
wandb: 	weight_decay: 0.0001
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: saleemkheralden (technionsaleem). Use `wandb login --relogin` to force relogin


batch loss,██▇▇▇▇▅█▆▆▆▅▇▅▄▅█▅▆▃▂▅▄▂▂▃▅▇▆▃▃▇▁▂▂▆▅▄▆▆
batch train accuracy,▂▂▃▁▂▁▄▂▅▄▄▅▂▄▄▅▂▄▂▆▆▄▄▆▇▆▅▁▄▆▇▃█▇▆▃▅▅▃▄
epoch,▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇███
loss,█▆▅▄▃▃▂▂▁▁
train accuracy,▁▂▄▅▆▆▇▇██
val accuracy,▁▁▂▃▄▃▇▆▇█
batch loss,0.48803
batch train accuracy,0.86957
epoch,9
loss,1.09355
train accuracy,0.63657


wandb: Agent Starting Run: 6oikmqwk with config:
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	learning_rate: 5e-05
wandb: 	n_samples: 256
wandb: 	optimizer: adam
wandb: 	weight_decay: 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


batch loss,█▇▇▇█▆▆▅▅▇▆▆▄▃▆▄▃▇▇▂▄▃▂▇▁▂▃▆▂▂▃▂▄▃▂▁▄▂▂▂
batch train accuracy,▃▄▃▃▁▄▃▄▅▂▄▄▇▆▃▅▆▃▃▆▄▇▆▂▇▆▇▅▇▆▆▇▆▇█▇▆▇▇▇
epoch,▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇███
loss,█▆▅▄▃▂▂▁▁▁
train accuracy,▁▃▄▅▆▇▇▇██
val accuracy,▁▄▇▃▇▆█▆▆▆
batch loss,0.72082
batch train accuracy,0.78261
epoch,9
loss,0.93495
train accuracy,0.69526


wandb: Agent Starting Run: 9k9r3rvk with config:
wandb: 	batch_size: 256
wandb: 	epochs: 10
wandb: 	learning_rate: 0.001
wandb: 	n_samples: 64
wandb: 	optimizer: adam
wandb: 	weight_decay: 0.0001
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

batch loss,█▇▆█▆▅█▄▄▄▃▇█▄▁▆▃█▅▃▂▆▂▂▃▅▃▁▆▄▂▂▂▁▃▅▁▃▁▂
batch train accuracy,▁▄▄▂▃▃▃▆▄▆▅▃▁▅█▂▆▂▄▆▆▄▇▇▆▅▇▇▃▆▇▇▇▇▆▅█▇██
epoch,▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇███
loss,█▆▄▃▂▂▂▁▁▁
train accuracy,▁▃▅▆▇▇▇███
val accuracy,▁▄▂▃█▆▃▆▆▅
batch loss,0.43004
batch train accuracy,0.82609
epoch,9
loss,0.80929
train accuracy,0.72269


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: u5p73yc3 with config:
wandb: 	batch_size: 256
wandb: 	epochs: 10
wandb: 	learning_rate: 0.001
wandb: 	n_samples: 128
wandb: 	optimizer: adam
wandb: 	weight_decay: 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


batch loss,█▇▇▆▆▆▆▅▅▇▆▅▇▇▆▆▄▄▄▅▄▃▂▄▂▄█▂▂▄▂▅▃▂▅▄▁▄▁▇
batch train accuracy,▁▂▂▃▄▃▄▄▄▂▂▄▂▂▅▂▅▄▄▄▅▆█▄▇▅▂▇▇▅█▅▆▇▅▆█▅▇▃
epoch,▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇███
loss,█▆▆▆▄▂▂▁▁▁
train accuracy,▁▃▃▄▅▇▇███
val accuracy,▁▁▁▂▃▄█▆▆▅
batch loss,2.08962
batch train accuracy,0.33113
epoch,9
loss,1.40999
train accuracy,0.51752


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 66jqdg9m with config:
wandb: 	batch_size: 256
wandb: 	epochs: 10
wandb: 	learning_rate: 0.0005
wandb: 	n_samples: 128
wandb: 	optimizer: adam
wandb: 	weight_decay: 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


batch loss,▇▇▇▆▆▇▆▆▅▅▆▆▆▆▄▅▅▅▃▄▃▇▅▄█▄▄▃▄▄▃▃▂▄▃▁▄▂▂▂
batch train accuracy,▁▂▂▂▃▃▅▄▄▄▃▃▄▃▅▄▃▄▆▆▆▅▄▆▃▅▆▆▅▆▆▆▆▅▆█▅█▇▇
epoch,▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇███
loss,█▇▆▅▄▃▄▃▂▁
train accuracy,▁▃▄▄▅▅▆▆▇█
val accuracy,▁▁▄▃▆▂▃▇▆█
batch loss,1.17581
batch train accuracy,0.59603
epoch,9
loss,1.27047
train accuracy,0.57876
